In [1]:
import numpy as np
import pandas as pd
import requests
import math

In [2]:
api_url = 'https://sandbox.iexapis.com/stable/'

with open('../secret.txt') as f:
    token = f.readline()
token = '?token=' + token

In [23]:
stocklist = ['AAPL', 'MSFT']
with open('sp_500_stocks.csv') as f:
    stocklist =  f.readlines()[1:]
stocklist = [s[:-1] for s in stocklist]
# get the list of them instead of statc

In [8]:
def get_dict(endpt, symbol):  # endpt = quote or advanced-stats, book etc.
    url = api_url + f'/stock/{symbol}/{endpt}'+ token
    res = requests.get(url)
    if res.status_code == 200:
        return res.json()
    return res.status_code


my_columns = [
              'symbol', 
              'latestPrice', 
              'marketCap', 
              'peRatio', 
              'beta', 
              'priceToBook', 
              'profitMargin', 
              'revenue'
             ]


def get_Series(symbol):
    A = get_dict('advanced-stats', symbol)
    Q = get_dict('quote', symbol)    
    
    A.update(Q)
    
    return pd.Series(
                {stat: A[stat] for stat in my_columns}
            )

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [5]:
DF = pd.DataFrame(columns= my_columns)

In [6]:
for symbol in stocklist:
    DF = DF.append(
            get_Series(symbol), ignore_index=True,
        )

In [7]:
DF

,symbol,latestPrice,marketCap,peRatio,beta,priceToBook,profitMargin,revenue
0,AAPL,126.10,2133724512590,28.89,1.432117,30.9,0.243779,332743900657
1,MSFT,248.37,1879327764605,33.98,1.238457,14.5,0.365197,160927281231


In [52]:
def get_batch(string):
    url = api_url + f'/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&' + token[1:]
    D = requests.get(url).json()
    L = []
    for res in D.values():
        d = {}
        for types in res.values():
            for k, v in types.items():
                d[k] = v
        L.append(d)
        
    return pd.DataFrame(L)

In [53]:
DFb = pd.DataFrame()
for sublist in chunks(stocklist, 100):
    symbol = ','.join(sublist)
    DFb = DFb.append(get_batch(symbol), ignore_index=True)
    
DFb

,EBITDA,avg10Volume,avg30Volume,avgTotalVolume,beta,calculationPrice,change,changePercent,close,closeSource,...,week52highSplitAdjustOnly,week52low,week52lowDate,week52lowDateSplitAdjustOnly,week52lowSplitAdjustOnly,year1ChangePercent,year2ChangePercent,year5ChangePercent,ytdChange,ytdChangePercent
0,1315279306.0,1046908.0,1350800.0,1325226.0,0.926382,iexlasttrade,-1.086,-0.00800,None,NaN,...,140.54,81.88,2020-05-10,2020-05-17,81.84,0.645934,0.944931,2.226318,0.104979,0.114713
1,-10839292686.0,36884079.0,36769706.0,38065622.0,1.650214,iexlasttrade,0.940,0.04313,None,NaN,...,27.03,9.82,2020-05-12,2020-05-07,9.55,1.516593,-0.31203,-0.28318,0.471853,0.4292
2,1035259534.0,722352.0,755640.0,735377.0,0.766287,iexlasttrade,1.914,0.00945,None,NaN,...,217.94,132.43,2020-05-17,2020-05-13,130.51,0.630419,0.310643,0.392668,0.299216,0.290008
3,103766304933.0,106938841.0,98738509.0,96714716.0,1.419071,iexlasttrade,-1.240,-0.00965,None,NaN,...,147.06,80.78,2020-05-09,2020-05-12,80.29,0.669293,1.806148,4.963677,-0.047858,-0.037693
4,21569170711.0,6590432.0,6224542.0,6509576.0,0.629008,iexlasttrade,0.490,0.00420,None,NaN,...,120.94,78.78,2020-10-15,2020-10-21,79.88,0.357504,0.648492,1.365027,0.120116,0.113888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,864935821,701987.0,677312.0,668579.0,0.940082,iexlasttrade,-0.740,-0.00600,None,NaN,...,124,61.58,2020-05-09,2020-05-04,60.21,1.094345,0.59829,1.956089,0.163648,0.169677
500,1964817883.0,1676070.0,1390957.0,1434593.0,0.675863,iexlasttrade,-0.590,-0.00490,None,NaN,...,124.47,86.88,2020-06-21,2020-06-19,87.06,0.442873,0.223106,1.227449,0.103970,0.104116
501,1517859289.0,1167991.0,1092961.0,1098501.0,1.109093,iexlasttrade,-1.410,-0.00833,None,NaN,...,187.16,114.11,2020-06-24,2020-06-18,116.0,0.505188,0.440469,0.456119,0.079906,0.09134
502,913504945.0,311213.0,322066.0,318200.0,1.312972,iexlasttrade,-4.600,-0.00964,None,NaN,...,542.02,229.53,2020-05-17,2020-05-07,228.01,1.256672,1.604253,8.868237,0.267304,0.275913


In [54]:
DFb[my_columns]

,symbol,latestPrice,marketCap,peRatio,beta,priceToBook,profitMargin,revenue
0,A,135.045,4.028256e+10,51.3,0.926382,8.62,0.150044,5533502103.0
1,AAL,23.410,1.545968e+10,-1.63,1.650214,-1.84,-0.622406,12899892349.0
2,AAP,211.057,1.363885e+10,29.21,0.766287,3.75,0.050564,10271328001.0
3,AAPL,129.232,2.121348e+12,28.95,1.419071,31.47,0.24143,328340027581.0
4,ABBV,122.100,2.148547e+11,40.93,0.629008,16.0,0.105372,52362193044.0
...,...,...,...,...,...,...,...,...
499,XYL,118.800,2.125843e+10,71.46,0.940082,7.5,0.0626,5156295266
500,YUM,121.300,3.685912e+10,31.91,0.675863,-4.6,0.198042,6090177020.0
501,ZBH,171.630,3.522515e+10,63.06,1.109093,2.9,0.080178,7314283529.0
502,ZBRA,486.910,2.667123e+10,40.4,1.312972,11.2,0.137099,4838406631.0


In [62]:
DF = DFb[my_columns]

In [63]:
# strategy here
DF['PosPerc'] = 1/len(DFb)

d:\work\algo-trade-tut\algo\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [69]:
investment = float(input('Enter Investment Size: '))

Enter Investment Size: 10000000


In [70]:
DF['posSize'] = investment*DF.PosPerc
DF['Sharetobebought'] = np.floor(DF.posSize/DF.latestPrice)

d:\work\algo-trade-tut\algo\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
d:\work\algo-trade-tut\algo\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [71]:
DF

,symbol,latestPrice,marketCap,peRatio,beta,priceToBook,profitMargin,revenue,PosPerc,posSize,Sharetobebought
0,A,135.045,4.028256e+10,51.3,0.926382,8.62,0.150044,5533502103.0,0.001984,19841.269841,146.0
1,AAL,23.410,1.545968e+10,-1.63,1.650214,-1.84,-0.622406,12899892349.0,0.001984,19841.269841,847.0
2,AAP,211.057,1.363885e+10,29.21,0.766287,3.75,0.050564,10271328001.0,0.001984,19841.269841,94.0
3,AAPL,129.232,2.121348e+12,28.95,1.419071,31.47,0.24143,328340027581.0,0.001984,19841.269841,153.0
4,ABBV,122.100,2.148547e+11,40.93,0.629008,16.0,0.105372,52362193044.0,0.001984,19841.269841,162.0
...,...,...,...,...,...,...,...,...,...,...,...
499,XYL,118.800,2.125843e+10,71.46,0.940082,7.5,0.0626,5156295266,0.001984,19841.269841,167.0
500,YUM,121.300,3.685912e+10,31.91,0.675863,-4.6,0.198042,6090177020.0,0.001984,19841.269841,163.0
501,ZBH,171.630,3.522515e+10,63.06,1.109093,2.9,0.080178,7314283529.0,0.001984,19841.269841,115.0
502,ZBRA,486.910,2.667123e+10,40.4,1.312972,11.2,0.137099,4838406631.0,0.001984,19841.269841,40.0


In [72]:
DF.to_csv('first.csv')